In [1]:
!pip install --upgrade transformers
!pip install --upgrade datasets
!pip install --upgrade huggingface_hub
!pip install evaluate
!pip install accelerate
!pip install ptflops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: 

In [2]:
import torch
from datasets import load_dataset
from huggingface_hub import login
from transformers import AutoImageProcessor, AutoFeatureExtractor
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)
from torch.utils.data import DataLoader
from transformers import ResNetForImageClassification, EfficientNetForImageClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

In [5]:
def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
val_ds = load_dataset('Tsomaros/Imagenet-1k_validation')

In [10]:
model_name = 'google/efficientnet-b7'
model_id = 'efficientnet-b7'
image_res = (3,600,600)
processor = AutoImageProcessor.from_pretrained(model_name)
model = EfficientNetForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
size = processor.size["height"]

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

image_mean, image_std = processor.image_mean, processor.image_std

normalize = Normalize(mean=image_mean, std=image_std)

_val_transforms = Compose(
    [
        Resize(size),
        CenterCrop(size),
        ToTensor(),
        normalize,
    ]
)

val_ds.set_transform(val_transforms)

In [16]:
args = TrainingArguments(
    f'{model_id}',
    evaluation_strategy="epoch",
    per_device_eval_batch_size=1,
    logging_dir='logs',
    remove_unused_columns=False,
    do_eval=True
)

trainer = Trainer(
    model,
    args,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  eval_train_accuracy           =     0.8329
  eval_train_loss               =     0.8079
  eval_train_runtime            = 1:08:49.99
  eval_train_samples_per_second =     12.107
  eval_train_steps_per_second   =     12.107


In [19]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(model, image_res, as_strings=False, print_per_layer_stat=False, verbose=False)
macs = round(int(macs) / 1000000000, 1)
params = round(int(params) / 1000000, 1)

print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       37.1    
Number of parameters:           66.3    


In [20]:
import json

filename = f'{model_id}' + '/eval_results.json'

with open(filename) as fp:
  dictObj = json.load(fp)

dictObj.update({"parameters": params,
                "GMacs": macs,
                "Device": torch.cuda.get_device_name(0),
                "Model Id": model_id})

In [21]:
with open(filename, 'w') as json_file:
    json.dump(dictObj, json_file,
                        indent=4,
                        separators=(',',': '))

In [22]:
from google.colab import drive
drive.mount('/content/drive')

%mkdir /content/drive/MyDrive/imagenet-1k/{model_id}
%cp /content/{model_id}/eval_results.json /content/drive/MyDrive/imagenet-1k/{model_id}

Mounted at /content/drive
